SQL-Extension. Muss ggf. mit `pip install ipython-sql` installiert werden.

In [1]:
%load_ext sql

Mit DB verbinden.

In [2]:
%sql sqlite:///sensordata.db

Ausschnitt der beiden wichtigen Views

In [14]:
%%sql 
select * 
from temp_hum_view 
limit 5

 * sqlite:///sensordata.db
Done.


year,month,day,hour,minute,temperature,humidity
2017,6,21,11,47,26.4,54.1
2017,6,21,11,50,26.6,50.2
2017,6,21,11,52,26.7,50.1
2017,6,21,11,55,26.9,49.8
2017,6,21,11,57,27.0,50.0


In [15]:
%%sql 
select * 
from feinstaub_view 
limit 5

 * sqlite:///sensordata.db
Done.


year,month,day,hour,minute,P1,P2
2017,6,21,11,47,12.23,3.35
2017,6,21,11,50,3.1,2.7
2017,6,21,11,52,4.03,2.6
2017,6,21,11,55,5.97,2.5
2017,6,21,11,57,5.8,2.67


Ein wenig Statistik

In [16]:
%sql select count(*) from temp_hum_view

 * sqlite:///sensordata.db
Done.


count(*)
644474


Durchschnittstemperaturen

In [17]:
%sql select avg(temperature) from temp_hum_view

 * sqlite:///sensordata.db
Done.


avg(temperature)
13.321666040834089


In [7]:
%%sql 
select year,avg(temperature) 
from temp_hum_view 
group by year

 * sqlite:///sensordata.db
Done.


year,avg(temperature)
2017,15.181615616136742
2018,14.410804213023562
2019,13.899668565048922
2020,12.049424730164592
2021,5.777783551068852


In [18]:
%%sql 
select year,month,avg(temperature) 
from temp_hum_view 
where year=2020
group by year,month

 * sqlite:///sensordata.db
Done.


year,month,avg(temperature)
2020,1,7.130253766851689
2020,2,8.273431148373982
2020,3,9.460176256465953
2020,4,15.878210610717574
2020,5,16.109047422193356
2020,6,20.19452819188813
2020,9,17.94297035515117
2020,10,13.30224880040104
2020,11,11.522822614251933
2020,12,7.165300489891687


Höchstwerte

In [19]:
%%sql 
select max(temperature) as "Höchsttemperatur"
from temp_hum_view

 * sqlite:///sensordata.db
Done.


Höchsttemperatur
50.6


Wann wurde diese Temperatur erreicht?

In [20]:
%%sql 
select *
from temp_hum_view
where temperature = 
(
    select max(temperature) 
    from temp_hum_view 
)

 * sqlite:///sensordata.db
Done.


year,month,day,hour,minute,temperature,humidity
2019,7,25,12,10,50.6,42.1
2019,7,25,12,13,50.6,40.6
2019,7,25,12,18,50.6,38.3


Das gleiche für die tiefsten Temperaturen.

In [21]:
%%sql 
select min(temperature) as "Tiefsttemperatur"
from temp_hum_view

 * sqlite:///sensordata.db
Done.


Tiefsttemperatur
-9.7


In [12]:
%%sql 
select *
from temp_hum_view
where temperature = 
(
    select min(temperature) 
    from temp_hum_view 
)

 * sqlite:///sensordata.db
Done.


year,month,day,hour,minute,temperature,humidity
2021,2,12,6,54,-9.7,99.9
2021,2,12,6,56,-9.7,99.9
2021,2,12,6,59,-9.7,99.9
2021,2,12,7,1,-9.7,99.9


Wieviele Messungen im Jahr lagen jeweils in den 1% der höchsten 
gemessenen Temperaturen?

In [13]:
%%sql

select year, count(*) "Anzahl Messungen" from 
(
    -- 1% der heißesten Tage
    select *
    from temp_hum_view
    order by temperature desc
    limit ((select count(*) from temp_hum_view) / 100)
)
group by year


 * sqlite:///sensordata.db
Done.


year,Anzahl Messungen
2017,446
2018,3150
2019,2387
2020,461
